# EICU data analysis

Here we run some basic analysis of the quality of data in the Emergency Information Coordination Unit (EICU) Rapid Damage Assessment data for the 2015 East Coast Low event in the Hunter Valley. 

There are some 2000 records in the database, covering communities from the NSW Central Coast and Hunter Valley region. 

We intend to use this data in conjunction with high-resolution numerical weather prediction data from the event to derive damage or fragility functions, which can then be used in an impact-based foreacting workflow for improving the information provided back to emergency managers. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from os.path import join as pjoin
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("poster")
sns.set_palette("Set1")

In [ ]:
def percentage(df, field):
    # This returns the percentage of records in each category,
    # including missing values in the total number
    pct = (df[field].value_counts()/len(df[field]))*100
    return pct

In [ ]:
dataPath = r"X:\georisk\HaRIA_B_Wind\projects\impact_forecasting\data\exposure\NSW\April_2015_Impact_Assessment/"
dataFile = "Property_Damage_cleaned.csv"
df = pd.read_csv(pjoin(dataPath, dataFile), skipinitialspace=True)
df.describe()

In [ ]:
df.columns

Examine the completeness of the records. Here we show the percentage of each field with entries (we haven't yet checked the valididity of those entries).

In [ ]:
qdf = pd.DataFrame(100. - 100 * df.isnull().sum()/len(df))
qdf

The level of damage is recorded in the `EICU_Degdamage` field, while the damage to individual structural elements is under `roofdam`, `frmecollap`, `bldleaning`, `rackingdam` and `grndmovement`. There is also a field for water inundation (`waterinund`), and a field recording the depth of water inundation (`maxwatrlev`). These fields are completed to greater and lesser degrees:

In [ ]:
qdf.loc[['EICU_Degdamage', 'frmecollap', 'bldleaning', 
         'rackingdam', 'grndmovemt', 'waterinund', 
         'maxwatrlev']]

Most of the damage-related attributes are categorical:

In [ ]:
df.EICU_Degdamage.value_counts()

In [ ]:
percentage(df, 'EICU_Degdamage')

## Water-related damage

Let's start by looking through the water damage fields. Water inundation can occur through multiple mechanisms: water rising up through the floor level (flood inundation), water flowing down through a property (storm), water ingress through openings (e.g. windows, doors, damage to roof, etc.). 

In [ ]:
df.waterinund.value_counts()

In [ ]:
percentage(df, 'waterinund')

So less than 10% of surveyed buildings were identified as inundated. There's still just under 40% of the survey population that we have no information about the state of inundation.

In [ ]:
df.maxwatrlev.value_counts()

What if we look into the quality of the recorded information for inundated buildings? Here, we select all the buildings which have been identified as inundated (`waterinund='Yes'`), then look at the value counts of the categorical entries. There are a small number where there is inundation, but the `maxwatrlev` is recorded as 'Not Applicable'. 

In [ ]:
df[df.waterinund=='Yes']['maxwatrlev'].value_counts()

In [ ]:
percentage(df[df.waterinund=='Yes'],'maxwatrlev')

So just over 4% of buildings identified as inundated are given a maximum water level of "Not Applicable". The question to be asked is whether this is water ingress resulting in damage to linings (e.g. through ceilings), or an incorrectly recorded field.

Around 40% of buildings identified as having been inundated had water between 5 and 300mm deep through the building. We can only presume this is above floor level.

__Issue__: Is there a way to relate a forecast variable to the water level above floor height?

Is it possible to identify the cause of inundation? Where there is water inundation, what is the recorded roof damage? 22 buildings are identified as having water inundation and some level of roof damage:

In [ ]:
df[df.waterinund=='Yes']['roofdam'].value_counts()

In [ ]:
df[df.waterinund=='Yes']['frmecollap'].value_counts()

Is it possible to use the `eventtype` field to attribute the cause of damage to a building? Let's start with examining the value counts of unique values in this field:

In [ ]:
df.eventtype.value_counts()

The vast majority of entries recorded in the EICU data are listed as a flood event. This does seem unusual, given the event was an East Coast Low. So does this indicate the cause of damage? Unlikely, given over 1100 surveyed buildings were identified to have no water inundation, or not applicable. 

Is there any correspondence between `eventtype` and `waterinund`?

In [ ]:
df[df.eventtype=='Flood']['waterinund'].value_counts()

In [ ]:
df[df.eventtype=='Storm']['roofdam'].value_counts()

Even when we examine 'Storm' events, there's no correlation to damage type - there's no entry where the `eventtype` is 'Storm' and damage to the roof has been recorded. 

In [ ]:
df[df.eventtype=='Storm']['EICU_Degdamage'].value_counts()

Of the 188 entries identified as 'Storm' events, 164 have some level of damage (i.e. not 'No Damage'). 

What is required is a field that indicates the primary cause of damage to a building such as wind, rain, flood, hail, etc. or some combination of these. 

We could make an assumption that roof damage is caused by wind, but that's not always the case (what about a hail storm). Similarly, is water damage always caused by flooding? Or is there a likelihood that it was wind-driven rain leading to water ingress?

A recommendation we could make to the Emergency Services is to include a field identifying the primary hazard causing damage, which is a categorical field with a set number of options that can then allow us to attribute the cause of damage, which in turn allows us to refine the damage functions.

The __National Damage Assessment Data Set and Dictionary for Phase 2 Assessments Guideline__ (AFAC, 2016) could be an avenue to formalise the collection of the hazard type that drives damage. The NDA DSD contains mandatory fields for collection for a range of hazard events, but there is no field that records the physical mechanism causing damage to structures - i.e. whether it is wind, rain, flood, fire, etc. The Guideline includes an 'Event Type' field, but this is not sufficient to be able to derive damage (or fragility) functions suitable for implementation in impact-based forecasting workflows. For example, an event type of "Cyclone" may result in damage from winds, rainfall, or storm surge. A "Storm" event could entail wind damge, hail damage or flood damage (and may cover east coast lows or thunderstorms). 

A recommendation to emergency services would be to collect the additional contextual information on the primary physical  mechanism of damage. To achieve this, there are a number of actions required:

- [ ] Check with our EM end-users about the implementation of the NDA DSD in their post-disaster survey/rapid damage assessment activities. THis would guide us on the value of using the NDA DSD to drive the required changes to data collection.
- [ ] Identify an appropriate field, with corresponding values, that would best serve our needs
- [ ] Clearly articulate the benefits of collecting this additional data
- [ ] Identify with AFAC opportunities for revision of the NDA DSD


### General data quality 

Here's an example of an incorrectly completed field. The value of 'NSWTF1' is in fact a value in the `teamid` field, but it's been transposed into the `frmecollap` field. There appear to be some similar transpositions in other fields, so maybe it's only a minor hiccup in the extraction.

In [ ]:
df.frmecollap.value_counts()


### Just do some quick cleaning up of the data

There are a number of blank fields throughout the data where a value was not entered into the dataset by the assessor. We need to keep track of the missing data, as well as the entered data, so we will find all 'NaN' values in the dataset, and change these to 'Not given' so we can include them in subsequent analyses.

In [ ]:
df = df.fillna('Not given')

In [ ]:
df.count()

In [ ]:

damage = df.groupby('EICU_Degdamage').count()
damage.OBJECTID

In [ ]:
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.countplot(x="EICU_Degdamage", hue="waterinund", data=df, order=order)
ax.legend(title="Water Damage?", loc=1)
ax.set_yscale("log", nonposy='clip')
plt.xticks(rotation=10)
plt.title('Damage State and presence of water')
plt.xlabel('Damage state')

### Damage by roof type

Is there any relationship between roof type and sustained damage? We group the data by the damage state, then look at the breakdown by roof material. i.e. what percentage of destroyed houses have tile or metal sheeting roofs? 

In [ ]:
temp = []
for name, group in df.groupby('EICU_Degdamage'):
    numingroup = group['OBJECTID'].count()
    numroofs = group.groupby('roof').count()
    percroofs = numroofs['OBJECTID']/numingroup*100
    for keys in percroofs.keys():
        temp.append((name, keys, percroofs[keys]))

roofdamage = pd.DataFrame(temp, columns = ['Damage_state', 'Roof_type', 'Percent'])
roofdamage

In [ ]:
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.factorplot(x="Damage_state", y = 'Percent', hue="Roof_type", data=roofdamage, order=order, kind = 'bar', size = 10)
plt.xticks(rotation=10)
plt.title('Damage state and roof type')
plt.ylabel('Percent of damage category')
plt.xlabel('Damage state')

It doesn't appear that there is a clear relationship. The breakdown by roof material is (approximately) the same in each damage category, reflecting the overall population distribution of roof materials. 

### Habitability

Is the `Habitable` field of a reasonable quality? We plot the number of buildings in each combination of habitability and damage category. It appears that theres a good attribution of habitability against the damage state, but the combination of damage state and habitability in the `Habitable` field is confounding. It's worth noting the AFAC National Damage Assessment Data Set Dictionary indicates the `Habitable` field should be 'Habitable', 'Not Habitable' or 'Not stated'.

In [ ]:
order = ['No Damage', 'Minor', 'Moderate', 'Severe', 'Total']
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.countplot(x="EICU_Degdamage", hue="propstatus", data=df, order=order)
ax.legend(title="Habitable", loc=10, bbox_to_anchor=(1.4, 0.5))
ax.set_yscale("log", nonposy='clip')
plt.xticks(rotation=10)


Is it possible to link habitability to roof material?

In [ ]:
g = sns.factorplot(x="EICU_Degdamage", hue="roof", col="propstatus", data=df, kind="count", order=order, col_wrap=2)
g.axes[0].set_yscale("log", nonposy="clip")

## Damage state versus incident wind speed

Now we will compare the damage state to the incident wind speeds simulated in the high-resolution ensemble forecast for the event. We've extracted the maximum wind speed at all building locations for a couple of ensemble members, then included them in the data frame. We then plot the incident wind speed against the assigned damage category for each point, and see if there's any reasonable relationship.

In the first step, we multiply the incident wind speed by a factor of 1.6, which represents a conversion from 10-minute wind speed to a 3-second gust wind speed. 

In [ ]:
df['combined_alltimes_maxwind_stage4_ens12'] = df['combined_alltimes_maxwind_stage4_ens12'] * 1.6
df['combined_alltimes_maxwind_stage4_ens22'] = df['combined_alltimes_maxwind_stage4_ens22'] * 1.6

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degdamage", x = "combined_alltimes_maxwind_stage4_ens22", data = df, order = order)
sns.swarmplot(y = "EICU_Degdamage", x = "combined_alltimes_maxwind_stage4_ens22", data = df, order = order, color = "0.2")

So it doesn't look very good! The destroyed buildings were wxposed to an incident wind speed of about 25 m/s, but there were vastly more buildings exposed to higher wind speeds that suffered only minor impact, or no damage. And this is of the buildings that were surveyed. 

We can look at another ensemble member to see if this is consistent...

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degdamage", x = "combined_alltimes_maxwind_stage4_ens12", data = df, order = order)
sns.swarmplot(y = "EICU_Degdamage", x = "combined_alltimes_maxwind_stage4_ens12", data = df, order = order, color = "0.2")

The same result, albeit a slightly lower wind speed for the collection of destroyed buildings. This suggests that wind speed was not a significant factor in the damage sustained in this event, at least for the more severely destroyed buildings. 

But one thing that does stand out is the sudden onset of damage to buildings. This may be a feature of the event itself - i.e. the maximum winds were generally above about 20 m/s - or it may arise because of the vulnerability profile of building stock in the area. 

We now take a cursory look at the potential for a threshold of damage. We define a field that is 0 if the damage category is "No Damage" and 1 otherwise. Then do a basic regression plot of wind speed against damaged/not damaged.

In [ ]:
damaged = np.zeros(len(df))
damaged[df['EICU_Degdamage'].isin(['Destroyed - 76-100%', 
                                 'Severe Impact - 51-75%', 
                                 'Major Impact - 26-50%', 
                                 'Minor Impact - 1-25%'])] = 1

df['Damaged'] = damaged

sns.lmplot(y = "Damaged", x = "combined_alltimes_maxwind_stage4_ens12", data = df,logistic=True, )


This is explored in more detail in another notebook.

